In [1]:
import time
import numpy as np
import pinocchio as pin
import crocoddyl

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator

from cnt_plan_utils import SoloCntGen


solo import 


In [2]:
robot = Solo12Config.buildRobotWrapper()
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [21]:
n_eff = 4
m = pin.computeTotalMass(robot.model)
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

In [51]:
f = np.load("dat_file/bound.npz")
mom_opt, com_opt = f["mom_opt"], f["com_opt"]
# com_opt[:,2] -= 0.02

In [80]:
# initial and ter state
X_init = np.zeros(9)
X_init[0:3] = q0[0:3]
X_ter = X_init.copy()

# contact plan
st = 0.20 # step time 
sh = 0.15 # step height
sl = np.array([0.0,0.0,0]) # step length
n_steps = 4 # number of steps
T = st*(n_steps + 2)
dt = 2e-2

X_ter[0:3] += sl*(n_steps)
X_nom = np.zeros((9*int(T/dt)))
X_nom[2::9] = X_init[2]

cnt_planner = SoloCntGen(T, dt, 2)
# print(cnt_plan)

In [84]:
cnt_planner.reset(T, dt)
cnt_plan = cnt_planner.create_trot_plan(st, sl, n_steps)
ik_solver = cnt_planner.create_ik_step_costs(cnt_plan, sh, [1e+5, 1e+6])

In [85]:
ik_solver.ik.add_com_position_tracking_task(0, T, com_opt, 1e+4, "com_track_cost")
ik_solver.create_centroidal_task(mom_opt, 0, T, "mom_track_cost", 1e+4)
xs = ik_solver.optimize(x0, wt_xreg=3e-3)

In [86]:
for i in range(len(xs)):
    time.sleep(0.05)
    viz.display(xs[i][:robot.model.nq])

In [41]:
viz.viewer.jupyter_cell()

In [1]:
for i in range(len(xs)):
    q = xs[i][:robot.model.nq]
    v = xs[i][robot.model.nq:]
    pin.forwardKinematics(robot.model, robot.data, q, v)
    pin.updateFramePlacements(robot.model, robot.data)
#     print(pin.centerOfMass(robot.model, robot.data, q, v)[2], com_opt[i][2])
    pin.computeCentroidalMomentum(robot.model, robot.data)
    print(np.round(robot.data.hg, 2)[4], np.round(mom_opt[i][4],2))
#     print(np.array(pin.getFrameVelocity(robot.model, robot.data, robot.model.getFrameId("FL_FOOT"), pin.LOCAL_WORLD_ALIGNED))[0:3])

NameError: name 'xs' is not defined

In [ ]:
j = 0
tmp = com_opt[0][j]
err = 0
for i in range(1, len(com_opt)):
    tmp += dt*mom_opt[i][j]/m
    err += abs(com_opt[i][j] - tmp)/len(com_opt)

print(err)